In [ ]:
import pandas as pd
import sklearn as sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score

In [96]:
matches = pd.read_csv("matches.csv", index_col=0)

In [ ]:
matches.head()

In [ ]:
matches.shape

In [ ]:
matches["team"].value_counts()

In [ ]:
matches[matches["team"] == "Liverpool"].sort_values("date")

In [ ]:
matches["round"].value_counts()

In [ ]:
matches.dtypes

In [104]:
del matches["comp"]

In [105]:
del matches["notes"]

In [106]:
matches["date"] = pd.to_datetime(matches["date"])

In [107]:
matches["target"] = (matches["result"] == "W").astype("int")

In [ ]:
matches

In [109]:
matches["venue_code"] = matches["venue"].astype("category").cat.codes

In [110]:
matches["opp_code"] = matches["opponent"].astype("category").cat.codes

In [111]:
matches["hour"] = matches["time"].str.replace(":.+", "", regex=True).astype("int")

In [117]:
matches["day_code"] = matches["date"].dt.dayofweek

In [120]:
rf = RandomForestClassifier(n_estimators=50, min_samples_split=10, random_state=1)

In [121]:
train = matches[matches["date"] < '2022-01-01']

In [122]:
test = matches[matches["date"] > '2022-01-01']

In [123]:
predictors = ["venue_code", "opp_code", "hour", "day_code"]

In [ ]:
rf.fit(train[predictors], train["target"])

In [125]:
preds = rf.predict(test[predictors])

In [127]:
error = accuracy_score(test["target"], preds)

In [129]:
combined = pd.DataFrame(dict(actual=test["target"], predicted=preds))

In [ ]:
pd.crosstab(index=combined["actual"], columns=combined["predicted"])

In [ ]:
precision_score(test["target"], preds)

In [209]:
grouped_matches = matches.groupby("team")

In [210]:
group = grouped_matches.get_group("Manchester City").sort_values("date")

In [211]:
def rolling_averages(group, cols, new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed='left').mean()
    group[new_cols] = rolling_stats
    group = group.dropna(subset=new_cols)
    return group

In [ ]:
cols = ["gf", "ga", "sh", "sot", "dist", "fk", "pk", "pkatt"]
new_cols = [f"{c}_rolling" for c in cols]

rolling_averages(group, cols, new_cols)

In [213]:
matches_rolling = matches.groupby("team").apply(lambda x: rolling_averages(x, cols, new_cols))

In [ ]:
matches_rolling

In [215]:
matches_rolling = matches_rolling.droplevel('team')

In [ ]:
matches_rolling

In [238]:
matches_rolling.index = range(matches_rolling.shape[0])

In [239]:
def make_predictions(data, predictors):
    train = data[data["date"] < '2022-01-01']
    test = data[data["date"] > '2022-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    error = precision_score(test["target"], preds)
    return combined, error

In [263]:
combined, error = make_predictions(matches_rolling, predictors + new_cols)

In [268]:
combined = combined.merge(matches_rolling[["date", "team", "opponent", "result"]], left_index=True, right_index=True)

In [286]:
class MissingDict(dict):
    __missing__ = lambda self, key: key

map_values = {"Brighton and Hove Albion": "Brighton", "Manchester United": "Manchester Utd", "Newcastle United": "Newcastle Utd", "Tottenham Hotspur": "Tottenham", "West Ham United": "West Ham", "Wolverhampton Wanderers": "Wolves"} 
mapping = MissingDict(**map_values)

In [287]:
combined["new_team"] = combined["team"].map(mapping)

In [290]:
merged = combined.merge(combined, left_on=["date", "new_team"], right_on=["date", "opponent"])

In [ ]:
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] ==0)]["actual_x"].value_counts()